## Imports

In [1]:
import json

# For DataFrame
import pandas as pd
import numpy as np


Calling save_info.py


In [2]:

import save_info

Available sectors:
1. Information Technology
2. Consumer Discretionary
3. Communication
4. Financials
5. Health Care
6. Energy
7. Consumer Staples
8. Cash and/or Derivatives
9. Materials
10. Industrials
11. Real Estate
12. Utilities
Stock symbols in the Information Technology sector:
['AAPL', 'MSFT', 'NVDA', 'AVGO', 'ADBE', 'CRM', 'ACN', 'AMD', 'CSCO', 'INTC']
Saved to out/info/AAPL.json
Saved to out/info/MSFT.json
Saved to out/info/NVDA.json
Saved to out/info/AVGO.json
Saved to out/info/ADBE.json
Saved to out/info/CRM.json
Saved to out/info/ACN.json
Saved to out/info/AMD.json
Saved to out/info/CSCO.json
Saved to out/info/INTC.json


READ CSV


In [3]:
# Read the CSV file and select columns 0 and 2
df = pd.read_csv("iShares-Russell-3000-ETF_fund.csv", usecols=[0, 2])

# Extract the unique sectors from the selected columns
unique_sectors = df.iloc[:, 1].unique()

# Display the available sectors
print("Available sectors:")
for i, sector in enumerate(unique_sectors, 1):
    print(f"{i}. {sector}")
from save_info import selected_sector
# Allow the user to select a sector
selected_sector1=selected_sector

# Filter symbols for the selected sector
selected_symbols = df[df.iloc[:, 1] == selected_sector1].iloc[:, 0].head(10).tolist()



Available sectors:
1. Information Technology
2. Consumer Discretionary
3. Communication
4. Financials
5. Health Care
6. Energy
7. Consumer Staples
8. Cash and/or Derivatives
9. Materials
10. Industrials
11. Real Estate
12. Utilities


## Configuration

In [4]:
# List of stock symbols we need to run fundamental analysis on - any symbol added here must have the json file
# containing stock info from YF
SYMBOLS = selected_symbols
DATA_PATH = 'out/info'

# Dictionary to collect data to create a DF later
data = {
    'Symbol': [],
    'Name': [],
    'Industry': [],
    'EPS (fwd)': [],
    'P/E (fwd)': [],
    'PEG': [],
    'FCFY' : [],
    'PB': [],
    'ROE' : [],
    'P/S (trail)': [],
    'DPR' : [],
    'DY' : [],
    'CR' : [],
    'Beta': [],
    'Price': [],
    '52w Low': [],
    '52w High': []
    }

## Loads data from json files

In [5]:
def load_data(json_data):
    data['Symbol'].append(json_data['symbol'])
    data['Name'].append(json_data['longName'])
    data['Industry'].append(json_data['industry'])
    data['Price'].append(json_data['currentPrice'])

    # Could be that some indicators are not available; use NaN if this is the case
    
    if 'forwardEps' in json_data:
        data['EPS (fwd)'].append(json_data['forwardEps'])
    else:
        data['EPS (fwd)'].append(np.nan)
        
    if 'forwardPE' in json_data:
        data['P/E (fwd)'].append(json_data['forwardPE'])
    else:
        data['P/E (fwd)'].append(np.nan)
        
    if 'pegRatio' in json_data:
        data['PEG'].append(json_data['pegRatio'])
    else:
        data['PEG'].append(np.nan)

    if ('freeCashflow' in json_data) and ('marketCap' in json_data):
        fcfy = (json_data['freeCashflow']/json_data['marketCap']) * 100
        data['FCFY'].append(round(fcfy, 2))
    else:
        data['FCFY'].append(np.nan)

    if 'priceToBook' in json_data:
        data['PB'].append(json_data['priceToBook'])
    else:
        data['PB'].append(np.nan)

    if 'returnOnEquity' in json_data:
        data['ROE'].append(json_data['returnOnEquity'])
    else:
        data['ROE'].append(np.nan)
        
    if 'priceToSalesTrailing12Months' in json_data:
        data['P/S (trail)'].append(json_data['priceToSalesTrailing12Months'])
    else:
        data['P/S (trail)'].append(np.nan)

    data['DPR'].append(json_data['payoutRatio'] * 100)

    if 'dividendYield' in json_data:
        data['DY'].append(json_data['dividendYield'])
    else:
        data['DY'].append(0.0)

    if 'beta' in json_data:
        data['Beta'].append(json_data['beta'])
    else:
        data['Beta'].append(np.nan)

    if 'currentRatio' in json_data:
        data['CR'].append(json_data['currentRatio'])
    else:
        data['CR'].append(np.nan)

    if 'fiftyTwoWeekLow' in json_data:
        data['52w Low'].append(json_data['fiftyTwoWeekLow'])
    else:
        data['52w Low'].append(np.nan)
        
    if 'fiftyTwoWeekHigh' in json_data:    
        data['52w High'].append(json_data['fiftyTwoWeekHigh'])
    else:
        data['52w High'].append(np.nan)

## Loads stock data from json files

In [6]:
for symbol in SYMBOLS:
    # Specify the full path to load JSON data
    file_name = f'{DATA_PATH}/{symbol}.json'    
    try:
        # Open the file in read mode
        with open(file_name, 'r') as file:
            # Use json.load() to parse the JSON data from the file
            load_data(json.load(file))
    except FileNotFoundError:
        print(f"File '{file_name}' not found.")
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON data: {e}")
    except Exception as e:
        print(f"An error occurred: {e}")

## Create DF

In [7]:
# Create a DF using the dictionary
df = pd.DataFrame(data)

# Save any stocks with NaN values
df_exceptions = df[df.isna().any(axis=1)]

# Remove any stocks with NaN values
df=df.dropna()

# Reset index after dropping rows with NaN values
df.reset_index(drop=True, inplace=True)

# Add 52 week price range
df['52w Range'] = ((df['Price'] - df['52w Low'])/(df['52w High'] - df['52w Low']))*100

df_exceptions

,Symbol,Name,Industry,EPS (fwd),P/E (fwd),PEG,FCFY,PB,ROE,P/S (trail),DPR,DY,CR,Beta,Price,52w Low,52w High
4,ADBE,Adobe Inc.,Software - Infrastructure,18.59,31.7929,2.74,NaN,16.280474,0.35513,13.864494,0.0,0.0,1.343,1.33,591.03,318.6,633.89


## Adds styles to DF

In [8]:
def make_pretty(styler):
    # Column formatting
    styler.format({'EPS (fwd)': '${:.2f}', 'P/E (fwd)': '{:.2f}', 'PEG': '{:.2f}',
                   'FCFY': '{:.2f}%', 'PB' : '{:.2f}', 'ROE' : '{:.2f}', 'P/S (trail)': '{:.2f}',
                   'DPR': '{:.2f}%', 'DY': '{:.2f}%', 'CR' : '{:.2f}', 'Beta': '{:.2f}', '52w Low': '${:.2f}',
                   'Price': '${:.2f}', '52w High': '${:.2f}', '52w Range': '{:.2f}%'
                  })
    # Set the bar visualization
    styler.bar(subset = ['52w Range'], align = "mid", color = ["salmon", "cornflowerblue"])

    # Grid
    styler.set_properties(**{'border': '0.1px solid black'})

    # Set background gradients
    styler.background_gradient(subset=['EPS (fwd)'], cmap='Greens')
    styler.background_gradient(subset=['P/E (fwd)'], cmap='Greens')
    styler.background_gradient(subset=['PEG'], cmap='Greens')
    styler.background_gradient(subset=['FCFY'], cmap='Greens')
    styler.background_gradient(subset=['PB'], cmap='Greens')
    styler.background_gradient(subset=['ROE'], cmap='Greens')
    styler.background_gradient(subset=['P/S (trail)'], cmap='Greens')
    styler.background_gradient(subset=['DPR'], cmap='Greens')
    styler.background_gradient(subset=['DY'], cmap='Greens')
    styler.background_gradient(subset=['CR'], cmap='Greens')

    # No index
    styler.hide(axis='index')

    # Tooltips
    styler.set_tooltips(
        ttips, css_class='tt-add',
        props=[
            ('visibility', 'hidden'),
            ('position', 'absolute'),
            ('background-color', 'salmon'),
            ('color', 'black'),
            ('z-index', 1),
            ('padding', '3px 3px'),
            ('margin', '2px')
        ]
    )
    # Left text alignment for some columns
    styler.set_properties(subset=['Symbol', 'Name', 'Industry'], **{'text-align': 'left'})
    return styler

## Adds Tool Tips

In [9]:
def populate_tt(df, tt_data, col_name):
    stats = df[col_name].describe()
    
    per25 = round(stats.loc['25%'], 2)
    per50 = round(stats.loc['50%'], 2)
    per75 = round(stats.loc['75%'], 2)

    # Get position based on the column name
    pos = df.columns.to_list().index(col_name)
    
    for index, row in df.iterrows():
        pe = row[col_name]
        if pe == stats.loc['min']:
            tt_data[index][pos] = 'Lowest'
        elif pe == stats.loc['max']:
            tt_data[index][pos] = 'Hightest'
        elif pe <= per25:
            tt_data[index][pos] = '25% of companies under {}'.format(per25)
        elif pe <= per50:
            tt_data[index][pos] = '50% of companies under {}'.format(per50)
        elif pe <= per75:
            tt_data[index][pos] = '75% of companies under {}'.format(per75)
        else:
            tt_data[index][pos] = '25% of companies over {}'.format(per75)    

## Apply styles and tooltips

In [10]:
# Initialize tool tip data - each column is set to '' for each row
tt_data = [['' for x in range(len(df.columns))] for y in range(len(df))]

# Gather tool tip data for indicators
populate_tt(df, tt_data, 'EPS (fwd)')
populate_tt(df, tt_data, 'P/E (fwd)')
populate_tt(df, tt_data, 'PEG')
populate_tt(df, tt_data, 'FCFY')
populate_tt(df, tt_data, 'PB')
populate_tt(df, tt_data, 'ROE')
populate_tt(df, tt_data, 'P/S (trail)')
populate_tt(df, tt_data, 'DPR')
populate_tt(df, tt_data, 'DY')
populate_tt(df, tt_data, 'CR')

# Create a tool tip DF
ttips = pd.DataFrame(data=tt_data, columns=df.columns, index=df.index)

# Add table caption and styles to DF
df.style.pipe(make_pretty).set_caption('Fundamental Indicators').set_table_styles(
    [{'selector': 'th.col_heading', 'props': 'text-align: center'},
     {'selector': 'caption', 'props': [('text-align', 'center'),
                                       ('font-size', '11pt'), ('font-weight', 'bold')]}])

Symbol,Name,Industry,EPS (fwd),P/E (fwd),PEG,FCFY,PB,ROE,P/S (trail),DPR,DY,CR,Beta,Price,52w Low,52w High,52w Range
AAPL,Apple Inc.,Consumer Electronics,$6.53,28.51,5.48,2.84%,46.58,1.72,7.56,15.33%,0.01%,0.99,1.29,$186.19,$131.44,$199.62,80.30%
MSFT,Microsoft Corporation,Software - Infrastructure,$11.83,32.36,2.60,1.79%,12.89,0.39,13.03,26.36%,0.01%,1.66,0.88,$382.77,$230.68,$384.30,99.00%
NVDA,NVIDIA Corporation,Semiconductors,$18.67,29.11,0.51,1.05%,40.29,0.69,29.92,2.11%,0.00%,3.59,1.64,$543.50,$154.92,$546.00,99.36%
AVGO,Broadcom Inc.,Semiconductors,$50.66,21.33,2.02,2.66%,18.25,0.60,14.12,55.79%,0.02%,2.81,1.24,$1080.57,$559.11,$1151.82,87.98%
CRM,"Salesforce, Inc.",Software - Application,$8.73,30.26,1.43,4.51%,4.41,0.04,7.53,0.00%,0.00%,1.04,1.28,$264.13,$143.16,$268.36,96.62%
ACN,Accenture plc,Information Technology Services,$13.39,25.79,3.64,4.07%,8.12,0.27,3.35,43.10%,0.01%,1.33,1.20,$345.33,$242.80,$355.38,91.07%
AMD,"Advanced Micro Devices, Inc.",Semiconductors,$3.42,43.43,5.25,1.15%,4.36,0.00,10.85,0.00%,0.00%,2.19,1.70,$148.54,$67.18,$151.05,97.01%
CSCO,"Cisco Systems, Inc.",Communication Equipment,$3.67,13.69,2.41,8.01%,4.50,0.32,3.52,46.83%,0.03%,1.47,0.89,$50.24,$45.56,$58.19,37.05%
INTC,Intel Corporation,Semiconductors,$1.72,27.60,5.69,-2.24%,1.97,-0.02,3.79,75.26%,0.01%,1.53,0.99,$47.47,$24.73,$51.28,85.65%


## Description
1. __[EPS](https://www.investopedia.com/terms/e/eps.asp)__ (Earnings Per Share): portion of a company’s profit that is assigned to each share of its stock
2. __[P/E](https://www.investopedia.com/terms/p/price-earningsratio.asp)__ (Price to Earnings): relationship between the stock price of a company and its per-share earnings. It helps investors determine if a stock is undervalued or overvalued relative to others in the same sector.
3. __[PEG](https://www.investopedia.com/terms/p/pegratio.asp)}__ (Projected Earnings Growth): - calculated by dividing a stock’s P/E by its projected 12-month forward revenue growth rate. In general, a PEG lower than 1 is a good sign, and a PEG higher than 2 indicates that a stock may be overpriced
4. __[FCFY](https://www.investopedia.com/articles/fundamental-analysis/09/free-cash-flow-yield.asp)__ (Free Cash Flow Yield): a financial solvency ratio that compares the free cash flow per share a company is expected to earn against its market value per share. A lower ratio indicates a less attractive investment opportunity.
5. __[PB](https://www.investopedia.com/terms/p/price-to-bookratio.asp)__ (Price to Book): A ratio of 1 indicates the company's shares are trading in line with its book value. A P/B higher than 1 suggests the company is trading at a premium to book value, and lower than 1 indicates a stock that may be undervalued relative to the company's assets.
6. __[ROE](https://www.investopedia.com/terms/r/returnonequity.asp)__ (Return on Equity): provides a way for investors to evaluate how effectively a company is using its equity to generate profits. A higher ROE indicates a more efficient use of shareholder equity, which can lead to increased demand for shares and higher stock price, as well as increase in company's profits in the future.
7. __[P/S](https://www.investopedia.com/terms/p/price-to-salesratio.asp)__ (Price to Sales): determines the fair value of a stock by utilizing a company’s market capitalization and revenue. It shows how much the market values the company’s sales, which can be effective in valuing growth stocks that have yet to turn a profit or aren’t performing as expected due to a temporary setback.
8. __[DPR](https://www.investopedia.com/terms/d/dividendpayoutratio.asp)__ (Dividend Payment Ratio): ratio of the total amount of dividends paid out to shareholders relative to the net income of the company. _Reference:_ __[Dividend Payout Ratio Definition, Formula, and Calculation](https://www.investopedia.com/terms/d/dividendpayoutratio.asp)__
9. __[DY](https://www.investopedia.com/terms/d/dividendyield.asp)__ (Dividend Yield Ratio): ratio looks at the amount paid by a company in dividends every year relative to its share price. It is an estimate of the dividend-only return of a stock investment.
10. __[CR](https://www.investopedia.com/terms/c/currentratio.asp)__ (Current Ratio): measures a company's ability to pay off its current liabilities (payable within one year) with its current assets such as cash, accounts receivable, and inventories. The higher the ratio, the better the company's liquidity position.
11. __[Beta](https://www.investopedia.com/terms/b/beta.asp)__: is a measure of a stock's volatility in relation to the overall market. A stock that swings more than the market over time has a beta above 1.0. If a stock moves less than the market, the stock's beta is less than 1.0.
12. __52w Range__: a visualization to indicate which stocks are near their 52 week low and which are near their 52 week high. For example, 90% will indicate that the current price is very close to its 52 week high

### Reference:
1. __[Top 10 Fundamental Analysis Indicators for All Investors](https://investoracademy.org/top-10-fundamental-analysis-indicators-for-all-investors/)__
2. __[Five Key Financial Ratios for Stock Analysis](https://www.schwab.com/learn/story/five-key-financial-ratios-stock-analysis#:~:text=Learn%20how%20these%20five%20key,depth%20of%20perspective%20you%20need)__